In [ ]:
import keras
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

def encode(series):
    return pd.get_dummies(series.astype(str))

def mean_absolute_error(y_true, y_pred): 
    y_true, y_pred = np.array(np.squeeze(y_true)), np.array(np.squeeze(y_pred))
    return np.mean(np.abs((y_true - y_pred)))

class traininghist(keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data
        self.trainingloss = []
        self.trainingmape = []
        
    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, mape = self.model.evaluate(x, y, verbose=0)
        self.trainingloss.append(loss)
        self.trainingmape.append(mape)
        print('Training loss: {}, mape: {}\n'.format(loss, mape))

In [ ]:
cnnr_model = keras.models.load_model('cnnr_109hz2.h5')
cnnr_model.summary()
cnnc_model = keras.models.load_model('cnnc_109hz2_small5.h5')
cnnc_model.summary()

In [ ]:
from keras.models import Model

layer1_model = Model(inputs=model.input,outputs=model.layers[1].output)
layer1bn_model = Model(inputs=model.input,outputs=model.layers[2].output)
layer2_model = Model(inputs=model.input,outputs=model.layers[4].output)
layer2bn_model = Model(inputs=model.input,outputs=model.layers[5].output)
layer3_model = Model(inputs=model.input,outputs=model.layers[7].output)
layer3bn_model = Model(inputs=model.input,outputs=model.layers[8].output)
layerd_model = Model(inputs=model.input,outputs=model.layers[11].output)
weights = model.get_weights()

In [ ]:

features1 = np.loadtxt('vec_mat_features_swellex_0.01train_109hz2.csv',delimiter=",")
#features2 = np.loadtxt('vec_mat_features_swellex_0.01trainbb_163hz.csv',delimiter=",")
#features3 = np.loadtxt('vec_mat_features_swellex_0.01trainbb_232hz.csv',delimiter=",")
#features4 = np.loadtxt('vec_mat_features_swellex_0.01trainbb_385hz.csv',delimiter=",")
labels_unstand = np.loadtxt('vec_mat_rlabels_swellex_0.01trainbb.csv',delimiter=",")
labels_t = []

features_mat = np.array([[features1]])#,[features2],[features3],[features4]])
X_train = np.zeros((features1.shape[0],21,21,2))

for f in range(features_mat.shape[0]):
    print(f)
    features = features_mat[f,0,:,:]
    real = features[:,0::2]
    imag = features[:,1::2]
    
    for k in range(features.shape[0]):
        count = 0
        for i in range(21):
            for j in range(i,21):
                X_train[k,i,j,2*f] = real[k,count]
                X_train[k,i,j,2*f+1] = imag[k,count]
                
                if i!=j:
                    X_train[k,j,i,2*f] = X_train[k,i,j,2*f]
                    X_train[k,j,i,2*f+1] = -X_train[k,i,j,2*f+1]
                
                count = count + 1

#X_train[k,:,:,0] = X_train[k,:,:,0]/np.amax(np.abs(X_train[k,:,:,0]))
#X_train[k,:,:,1] = X_train[k,:,:,1]/np.amax(np.abs(X_train[k,:,:,1]))

labels_unstand = labels_unstand.ravel()
#y_train,mu,sigma_labels = std_y(labels_unstand)

y_train = labels_unstand


In [ ]:
layerd_output = layerd_model.predict(X_train)
np.savetxt('layerd_output_train_r.tsv',layerd_output,delimiter='\t')

In [ ]:
np.savetxt('layerd_output_train_rlabels.tsv',y_train,delimiter='\t')

In [ ]:
#Regression
#Import test data

features_test1 = np.loadtxt('vec_mat_features_swellex_109hz2_test2.csv',delimiter=",")
#features_test2 = np.loadtxt('vec_mat_features_swellex_test2_163hz2.csv',delimiter=",")
#features_test3 = np.loadtxt('vec_mat_features_swellex_test2_232hz2.csv',delimiter=",")
#features_test4 = np.loadtxt('vec_mat_features_swellex_test2_385hz2.csv',delimiter=",")
temp_ytest = np.loadtxt('vec_mat_rlabels_swellex_109hz2_test.csv',delimiter=",")
y_test= []

featurestest_mat = np.array([[features_test1]])#,[features_test2],[features_test3],[features_test4]])
X_test = np.zeros((features_test1.shape[0],21,21,2))

for f in range(featurestest_mat.shape[0]):
    print(f)
    features_test = featurestest_mat[f,0,:,:]
    real_test = features_test[:,0::2]
    imag_test = features_test[:,1::2]

    for k in range(features_test.shape[0]):
        count = 0
        for i in range(21):
            for j in range(i,21):
                X_test[k,i,j,2*f] = real_test[k,count]
                X_test[k,i,j,2*f+1] = imag_test[k,count]
            
                if i!=j:
                    X_test[k,j,i,2*f] = X_test[k,i,j,2*f]
                    X_test[k,j,i,2*f+1] = -X_test[k,i,j,2*f+1]
            
                count = count + 1

#X_test[k,:,:,0] = X_test[k,:,:,0]/np.amax(np.abs(X_test[k,:,:,0]))
#X_test[k,:,:,1] = X_test[k,:,:,1]/np.amax(np.abs(X_test[k,:,:,1]))

temp_ytest = temp_ytest.ravel()
#y_test = (temp_ytest - mu)/sigma_labels
y_test = temp_ytest
#y_test = y_test

print(X_test.shape)
print(y_test.shape)

In [ ]:
#Testing
y_test = y_test
test_loss, test_acc = model.evaluate(X_test,y_test)

predictions = model.predict(X_test)
diff = abs((np.transpose(predictions))-(y_test))
error = diff[diff>1]
percent_correct = (len(y_test)-len(error))/len(y_test)

print('Test mae:', test_acc)
print('Testing percent within 1km:', percent_correct*100)

In [ ]:

inds = y_test.argsort()
sorted_y_test = y_test[inds]

sorted_y_pred = np.squeeze(predictions)[inds]

#Plotting
colors = sorted_y_pred-sorted_y_test
norm = Normalize()
norm.autoscale(colors)
colormap = cm.rainbow
fig1 = plt.figure(figsize = (10,8))
plt.scatter(sorted_y_test,sorted_y_pred,marker='.',c='k',alpha=0.75)
plt.plot(sorted_y_test,sorted_y_test,'r',linewidth = 2)
plt.quiver(sorted_y_test, sorted_y_test, np.zeros(len(sorted_y_test)), sorted_y_pred-sorted_y_test,color=colormap(norm(colors)),angles='xy', scale_units='xy', scale=1.,width=0.002)
plt.axis([0, 10, 0, 10],fontsize=20)
plt.ylabel('Predicted Distance (km)',fontsize=20)
plt.xlabel('Actual Distance (km)',fontsize=20)
plt.title('CNN Regression Predictions (Depth = 216.5m)',fontsize=20)
plt.grid()

In [ ]:
fig1.savefig('cnnr_test9.png')

In [ ]:
#Classification

#Import label data
import pandas as pd
temp_labels = np.loadtxt('vec_mat_clabels_swellex_0.01trainbb.csv',delimiter=",")
labels_t = []

for l in range(len(temp_labels)):
    labels_t.append(str(temp_labels[l]))

labels_t = np.array(labels_t)
labels_t = labels_t.ravel()

def encode(series):
    return pd.get_dummies(series.astype(str))

y_train = encode(labels_t)
labels = list(y_train.columns.values)

y_train = pd.DataFrame.as_matrix(y_train)

#import test data
features_test = np.loadtxt('vec_mat_features_swellex_109hz2_test2.csv',delimiter=",")
temp_ytest = np.loadtxt('vec_mat_rlabels_swellex_109hz2_test.csv',delimiter=",")
y_test= []

real_test = features_test[:,0::2]
imag_test = features_test[:,1::2]
X_test = np.zeros((features_test.shape[0],21,21,2))

for k in range(features_test.shape[0]):
    count = 0
    for i in range(21):
        for j in range(i,21):
            X_test[k,i,j,0] = real_test[k,count]
            X_test[k,i,j,1] = imag_test[k,count]
            
            if i!=j:
                X_test[k,j,i,0] = X_test[k,i,j,0]
                X_test[k,j,i,1] = -X_test[k,i,j,1]
            
            count = count + 1

#X_test[k,:,:,0] = X_test[k,:,:,0]/np.amax(np.abs(X_test[k,:,:,0]))
#X_test[k,:,:,1] = X_test[k,:,:,1]/np.amax(np.abs(X_test[k,:,:,1]))


temp_ytest = temp_ytest.ravel()
#y_test = (temp_ytest - mu)/sigma_labels
y_test = temp_ytest
#y_test = y_test

#print(X_test.shape)
#print(y_test.shape)

predictions = model.predict(X_test)

pred_labels = []
for i in np.argmax(predictions, axis=1):
    pred_labels.append(float(labels[i]))
    
#print(np.shape(pred_labels))

#Classification acc
mae = mean_absolute_error(y_test, pred_labels)
diff = abs((np.transpose(pred_labels))-(y_test))
error = diff[diff>.1]
percent_correct = (len(y_test)-len(error))/len(y_test)

print('Test mae:', mae)
print('Testing percent within 1km:', percent_correct*100)

In [ ]:
#sort output for plotting

#floaty_test = label_test.astype(np.float)
inds = y_test.argsort()
sorted_y_test = y_test[inds]

floaty_pred = np.array(pred_labels).astype(np.float)
sorted_y_pred = floaty_pred[inds]


fig, ax = plt.subplots(figsize=(10,6))

c = ax.pcolormesh(sorted_y_test, np.linspace(0,10,101), predictions.T*100, cmap='jet',vmin=0, vmax=100)
ax.plot([0,10],[0,10],'g--')
ax.set_title('CNN Classification Result ',fontsize=20)
ax.set_ylabel('Predicted Source Distance (km)',fontsize=20)
ax.set_xlabel('Actual Source Distance (km)',fontsize=20)
ax.axis([0,10, 0, 10],fontsize=20)
# set the limits of the plot to the limits of the data
#ax.axis([x.min(), x.max(), y.min(), y.max()])
fig.colorbar(c, ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
sorted_y_pred = np.array([float(numeric_string) for numeric_string in pred_labels])
sorted_y_test = np.array([float(numeric_string) for numeric_string in y_test])

colors = sorted_y_pred-sorted_y_test
norm = Normalize()
norm.autoscale(colors)
colormap = cm.rainbow
fig1 = plt.figure(figsize = (10,8))
plt.scatter(sorted_y_test,sorted_y_pred,marker='.',c='k',alpha=0.75)
plt.plot(sorted_y_test,sorted_y_test,'r',linewidth = 2)
plt.quiver(sorted_y_test, sorted_y_test, np.zeros(len(sorted_y_test)), sorted_y_pred-sorted_y_test,color=colormap(norm(colors)),angles='xy', scale_units='xy', scale=1.,width=0.002)
plt.axis([0, 10, 0, 10],fontsize=20)
plt.ylabel('Predicted Distance (km)',fontsize=20)
plt.xlabel('Actual Distance (km)',fontsize=20)
plt.title('CNN Classification Test Predictions',fontsize=20)
plt.grid()

In [ ]:
#MFP test

mfp_pred = np.loadtxt('mfp_test7_pred.csv',delimiter=",")
mfp_act = np.loadtxt('vec_mat_rlabels_swellex_109hz2_test.csv',delimiter=",")

mae = mean_absolute_error(mfp_act, mfp_pred)
print('Test mae:', mae)

diff = abs((np.transpose(mfp_pred))-(mfp_act))
error = diff[diff>1]
percent_correct = (len(mfp_act)-len(error))/len(mfp_act)
print('Percent within 1km:', percent_correct*100)

In [ ]:
#Plotting
colors = mfp_pred-mfp_act
norm = Normalize()
norm.autoscale(colors)
colormap = cm.rainbow
fig1 = plt.figure(figsize = (10,8))
plt.scatter(mfp_act,mfp_pred,marker='.',c='k',alpha=0.75)
plt.plot(mfp_act,mfp_act,'r',linewidth = 2)
plt.quiver(mfp_act, mfp_act, np.zeros(len(mfp_act)), mfp_pred-mfp_act,color=colormap(norm(colors)),angles='xy', scale_units='xy', scale=1.,width=0.002)
plt.axis([0, 10, 0, 10],fontsize=20)
plt.ylabel('Predicted Distance (km)',fontsize=20)
plt.xlabel('Actual Distance (km)',fontsize=20)
plt.title('MFP Predictions (no SSP mismatch)',fontsize=20)
plt.grid()

In [ ]:
fig1.savefig('cnnr_test9.png')

In [ ]:

#Import Real test data

features_test11 = np.loadtxt('vec_mat_features_swellex_shallow_109hz2.csv',delimiter=",")
#features_test22 = np.loadtxt('vec_mat_features_swellex_shallow_163hz.csv',delimiter=",")
#features_test33 = np.loadtxt('vec_mat_features_swellex_shallow_232hz.csv',delimiter=",")
#features_test44 = np.loadtxt('vec_mat_features_swellex_shallow_385hz.csv',delimiter=",")
#temp_ytest = np.loadtxt('vec_mat_rlabels_swellex_test2.csv',delimiter=",")
#y_test= []

featurestest_mat1 = np.array([[features_test11]])#,[features_test22],[features_test33],[features_test44]])
X_test1 = np.zeros((features_test11.shape[0],21,21,2))

for f in range(featurestest_mat1.shape[0]):
    print(f)
    features_test0 = featurestest_mat1[f,0,:,:]
    real_test = features_test0[:,0::2]
    imag_test = features_test0[:,1::2]

    for k in range(features_test0.shape[0]):
        count = 0
        for i in range(21):
            for j in range(i,21):
                X_test1[k,i,j,2*f] = real_test[k,count]
                X_test1[k,i,j,2*f+1] = imag_test[k,count]
            
                if i!=j:
                    X_test1[k,j,i,2*f] = X_test1[k,i,j,2*f]
                    X_test1[k,j,i,2*f+1] = -X_test1[k,i,j,2*f+1]
            
                count = count + 1

#X_test1[k,:,:,0] = X_test1[k,:,:,0]/np.amax(np.abs(X_test1[k,:,:,0]))
#X_test1[k,:,:,1] = X_test1[k,:,:,1]/np.amax(np.abs(X_test1[k,:,:,1]))

#temp_ytest = temp_ytest.ravel()
#y_test = (temp_ytest - mu)/sigma_labels
#y_test = temp_ytest
#y_test = y_test+5

print(X_test1.shape)
#print(y_test.shape)


dist_actual = np.loadtxt('dist_actual.csv')
np.shape(dist_actual)

In [ ]:
#interpolate dist_actual
from scipy import interpolate

tck = interpolate.splrep(np.arange(77), dist_actual, s=0)
xnew = np.linspace(0,77,X_test1.shape[0])
ynew = interpolate.splev(xnew, tck, der=0)

In [ ]:
#import mfp regression output

mfp_output = np.loadtxt('mfp_109hz2_output2.csv')
#mfp_output = np.round(mfp_output,1)
print(mfp_output.shape)
time = np.linspace(0,75,mfp_output.shape[0])

colors = mfp_output-ynew
norm = Normalize()
norm.autoscale(colors)
colormap = cm.rainbow
fig1 = plt.figure(figsize = (10,8))
plt.scatter(xnew,mfp_output,marker='.',c='k',alpha=0.75)
plt.plot(xnew,ynew,'r',linewidth = 2)
plt.quiver(xnew, ynew, np.zeros(len(xnew)), mfp_output-ynew,color=colormap(norm(colors)),angles='xy', scale_units='xy', scale=1.,width=0.002)
plt.axis([0,75, 0, 10],fontsize=20)
plt.ylabel('Predicted Distance (km)',fontsize=20)
plt.xlabel('Time (min)',fontsize=20)
plt.title('MFP Predictions',fontsize=20)
plt.grid()

#plt.plot(time,mfp_output,'g*')
##plt.plot(np.arange(77),dist_actual,'r')
#plt.axis([0, 77, 0, 10],fontsize=20)

In [ ]:
fig1.savefig('swellex_mfp_pred.png')

In [ ]:
#regression Test

predictionsr = cnnr_model.predict(X_test1)
time = np.linspace(0,75,X_test1.shape[0])
predictionsr = np.squeeze(np.array(predictionsr))
colors = predictionsr-ynew
norm = Normalize()
norm.autoscale(colors)
colormap = cm.rainbow
fig1 = plt.figure(figsize = (10,8))
plt.scatter(xnew,predictionsr,marker='.',c='k',alpha=0.75)
plt.plot(xnew,ynew,'r',linewidth = 2)
plt.quiver(xnew, ynew, np.zeros(len(xnew)), predictionsr-ynew,color=colormap(norm(colors)),angles='xy', scale_units='xy', scale=1.,width=0.002)
plt.axis([0,75, 0, 10],fontsize=20)
plt.ylabel('Predicted Distance (km)',fontsize=20)
plt.xlabel('Time (min)',fontsize=20)
plt.title('CNN Regression Predictions',fontsize=20)
plt.grid()


#plt.plot(time,predictions1,'o')
#plt.plot(time,mfp_output,'g*')
#plt.plot(np.arange(77),dist_actual,'r')
#plt.axis([0, 77, 0, 10],fontsize=20)

In [ ]:
fig1.savefig('swellex_cnnr_pred.png')

In [ ]:
#Import label data
import pandas as pd
temp_labels = np.loadtxt('vec_mat_clabels_swellex_0.01trainbb.csv',delimiter=",")
labels_t = []

for l in range(len(temp_labels)):
    labels_t.append(str(temp_labels[l]))

labels_t = np.array(labels_t)
labels_t = labels_t.ravel()

def encode(series):
    return pd.get_dummies(series.astype(str))

y_train = encode(labels_t)
labels = list(y_train.columns.values)

y_train = pd.DataFrame.as_matrix(y_train)


#Classification Test
time = np.linspace(0,75,X_test1.shape[0])
predictions1 = cnnc_model.predict(X_test1)
predictions1.shape
pred_labels = []
for i in np.argmax(predictions1, axis=1):
    pred_labels.append(float(labels[i]))
 
colors = pred_labels-ynew
norm = Normalize()
norm.autoscale(colors)
colormap = cm.rainbow
fig1 = plt.figure(figsize = (10,8))
plt.scatter(xnew,pred_labels,marker='.',c='k',alpha=0.75)
plt.plot(xnew,ynew,'r',linewidth = 2)
plt.quiver(xnew, ynew, np.zeros(len(xnew)), pred_labels-ynew,color=colormap(norm(colors)),angles='xy', scale_units='xy', scale=1.,width=0.002)
plt.axis([0,75, 0, 10],fontsize=20)
plt.ylabel('Predicted Distance (km)',fontsize=20)
plt.xlabel('Time (min)',fontsize=20)
plt.title('CNN Classification Predictions',fontsize=20)
plt.grid()


#plt.plot(time,pred_labels,'o')
#plt.plot(np.arange(77),dist_actual,'r')
#plt.axis([0, 77, 0, 10],fontsize=20)
#plt.grid
#print(np.shape(pred_labels))

In [ ]:
fig1.savefig('swellex_cnnc_pred.png')

In [ ]:
#Plotting
fig = plt.figure(figsize=(20,6))
fig.suptitle('Swellex96 Source Range Predictions', fontsize=20, fontweight='bold')
fig.text(0.09, 0.5, 'Predicted Distance (Km)', fontsize=20,va='center', rotation='vertical')

plt.subplot(1,3,1)
colors = mfp_output-ynew
norm = Normalize()
norm.autoscale(colors)
colormap = cm.rainbow
plt.scatter(xnew,mfp_output,marker='.',c='k',alpha=0.75)
plt.plot(xnew,ynew,'r',linewidth = 2)
plt.quiver(xnew, ynew, np.zeros(len(ynew)), mfp_output-ynew,color=colormap(norm(colors)),angles='xy', scale_units='xy', scale=1.,width=0.002)
plt.axis([0, 75, 0, 10],fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
#plt.ylabel('Predicted Distance (Km)',fontsize=20)
#plt.xlabel('Actual Distance (Km)',fontsize=20)
plt.title('MFP',fontsize=20)
plt.grid()

plt.subplot(1,3,2)
colors = pred_labels-ynew
norm = Normalize()
norm.autoscale(colors)
colormap = cm.rainbow
plt.scatter(xnew,pred_labels,marker='.',c='k',alpha=0.75)
plt.plot(xnew,ynew,'r',linewidth = 2)
plt.quiver(xnew, ynew, np.zeros(len(ynew)), pred_labels-ynew,color=colormap(norm(colors)),angles='xy', scale_units='xy', scale=1.,width=0.002)
plt.axis([0, 75, 0, 10],fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Time (min)',fontsize=20)
plt.title('CNN-c',fontsize=20)
plt.grid()

plt.subplot(1,3,3)
colors = predictionsr-ynew
norm = Normalize()
norm.autoscale(colors)
colormap = cm.rainbow
plt.scatter(xnew,predictionsr,marker='.',c='k',alpha=0.75)
plt.plot(xnew,ynew,'r',linewidth = 2)
plt.quiver(xnew, ynew, np.zeros(len(ynew)), predictionsr-ynew,color=colormap(norm(colors)),angles='xy', scale_units='xy', scale=1.,width=0.002)
plt.axis([0, 75, 0, 10],fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
#plt.xlabel('Actual Distance (km)',fontsize=20)
plt.title('CNN-r',fontsize=20)
plt.grid()

plt.savefig('/Users/Rui/Desktop/swellex_test.eps')

plt.show()

In [ ]:
#Regression acc
mae = mean_absolute_error(ynew, predictions1)
diff = abs((np.transpose(predictions1))-(ynew))
error = diff[diff>1]
percent_correct = (len(ynew)-len(error))/len(ynew)

print('Test mae:', mae)
print('Testing percent within 1km:', percent_correct*100)

In [ ]:
#Classification acc
mae = mean_absolute_error(ynew, pred_labels)
diff = abs((np.transpose(pred_labels))-(ynew))
error = diff[diff>.1]
percent_correct = (len(ynew)-len(error))/len(ynew)

print('Test mae:', mae)
print('Testing percent within 1km:', percent_correct*100)

In [ ]:
#mfp acc
#mfp_output = np.round(mfp_output,1)
diff = abs((np.transpose(mfp_output))-(ynew))
error = diff[diff>1]
percent_correct = (len(ynew)-len(error))/len(ynew)
mape = mean_absolute_error(ynew, mfp_output)
print('Test mae:', mape)
print('Testing percent within 1km:', percent_correct*100)